## Source: [gazet_dataset/load_dataset.ipynb](https://github.com/mirmashel/gazet_dataset/blob/main/load_dataset.ipynb)

In [1]:
import json
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import shutil
import pandas as pd

data_dir = "/mnt/hdd/ai_n_slo/gazet-data-original/dataset"
destination_directory = "/mnt/hdd/ai_n_slo/gazet-data-cropped"

In [2]:
# load metadata and index
with open(os.path.join(data_dir, 'metadata.json'), 'r') as f:
    meta_index = json.load(f)

In [3]:
def load_image(meta):
    '''
    Read images
    '''
    image_path = os.path.join(data_dir, 'images', meta['task_id'], meta['step'])
    return cv2.imread(image_path)[..., ::-1]

In [4]:
def draw_meta(image, meta):
    '''
    Visualize metadata, draw eyes points and gaze point on the screen
    '''
    result_image = np.array(image)
    image_size = image.shape[:2]
    # draw eyes points 
    for point in [
        meta['eyes_left_left'],
        meta['eyes_left_right'],
        meta['eyes_right_left'],
        meta['eyes_right_right']
    ]:
        result_image = cv2.circle(result_image, point, max(2, int(image_size[0] * 0.01)), (0, 255, 0), -1)

    # visualize point of gaze on the screen
    screen = np.zeros((meta['screen_size_y'], meta['screen_size_x'], 3), dtype=np.uint8)
    gaze = [int(meta['screen_size_x'] * (meta['relative_x'])), int(meta['screen_size_y'] * meta['relative_y'])]
    screen = cv2.circle(screen, gaze, max(2, int(meta['screen_size_x'] * 0.02)), (255, 0, 0), -1)
    return result_image, screen

In [5]:
def crop_image(image, bbox):
    '''
    Crop bbox from image
    '''
    x, y, w, h = bbox
    w_matrix = [1, 0, float(x)]
    h_matrix = [0, 1, float(y)]

    image = cv2.warpAffine(
        image,
        np.column_stack((w_matrix, h_matrix)).T,
        (w, h),
        flags=cv2.INTER_LINEAR | cv2.WARP_INVERSE_MAP,
        borderMode=cv2.BORDER_REPLICATE,
    )
    return image

# create bounding box of eyes field and crop it from image to further train
def crop_eyes(image, meta):
    '''
    Create bounding box from eyes points and crop eyes field from image to further train and inference
    '''
    left_left_point, left_right_point, right_left_point, right_right_point = [
        meta['eyes_left_left'],
        meta['eyes_left_right'],
        meta['eyes_right_left'],
        meta['eyes_right_right']
    ]
    left_med = [(left_left_point[0] + left_right_point[0]) / 2, (left_left_point[1] + left_right_point[1]) / 2]
    right_med = [(right_left_point[0] + right_right_point[0]) / 2, (right_left_point[1] + right_right_point[1]) / 2]

    left_dist = np.sqrt(
        (left_left_point[0] - left_right_point[0]) ** 2 + (left_left_point[1] - left_right_point[1]) ** 2
    )
    right_dist = np.sqrt(
        (right_left_point[0] - right_right_point[0]) ** 2 + (right_left_point[1] - right_right_point[1]) ** 2
    )
    mean_dist = (left_dist + right_dist) / 2

    cropped_image = crop_image(
        image,
        bbox=[
            int(min(left_med[0] - mean_dist, right_med[0] - mean_dist)),
            int(min(left_med[1] - mean_dist, right_med[1] - mean_dist)),
            int(2 * mean_dist + abs(left_med[0] - right_med[0])),
            int(2 * mean_dist + abs(left_med[1] - right_med[1])),
        ],
    )
    return cropped_image

In [6]:
os.makedirs(os.path.join(destination_directory, "images"))
shutil.copy2(os.path.join(data_dir, 'metadata.json'), destination_directory)

image_sizes = []

for i_, meta in enumerate(meta_index):
    image = load_image(meta)
    
    h, w = image.shape[:2]
    image_sizes.append({
        "task_id": meta["task_id"],
        "step": meta["step"],
        "h": h,
        "w": w
    })
    
    # image_with_meta, screen = draw_meta(image, meta)
    image_crop = crop_eyes(image, meta)

    task_dir = os.path.join(destination_directory, 'images', meta['task_id'])
    if not os.path.exists(task_dir):
        os.makedirs(task_dir)

    image_crop = cv2.cvtColor(image_crop, cv2.COLOR_BGR2RGB)

    cv2.imwrite(os.path.join(destination_directory, 'images', meta['task_id'], meta['step']), image_crop)

    if i_%10_000 == 0:
        print(f"[{i_}/{len(meta_index)}]")
    
pd.DataFrame(image_sizes).to_csv(os.path.join(destination_directory, "image_sizes.csv"), sep="\t")

[0/144275]
[10000/144275]
[20000/144275]
[30000/144275]
[40000/144275]
[50000/144275]
[60000/144275]
[70000/144275]
[80000/144275]
[90000/144275]
[100000/144275]
[110000/144275]
[120000/144275]
[130000/144275]
[140000/144275]
